In [5]:
# Import necessary packages
import numpy as np
from skimage import data
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops_table
from skimage.morphology import closing, square, remove_small_objects
import napari

### Segment ROIs ###

In [34]:
def segment(image):
    """Segment an image using an intensity threshold
    determined via Otsu's method.

    Parameters
    ----------
    image : np.ndarray
        The image to be segmented

    Returns
    -------
    label_image : np.ndarray
        The resulting image where each detected object
        is labeled with a unique integer.
    """
    # apply threshold
    thresh = threshold_otsu(image)
    bw = closing(image > thresh, square(2))

    # remove artifacts connected to image border
    cleared = remove_small_objects(clear_border(bw), 25)

    # label image regions
    label_image = label(cleared)

    return label_image

In [35]:
# Create an empty viewer
viewer = napari.Viewer()

In [36]:
image = imread('/Users/virly/beta_script/vesicle-detection/vesicles.png')

viewer.add_image(image)

<Image layer 'image' at 0x7f89c8c245b0>

In [37]:
# Create label
label_image = segment(image)

# initialize viewer with coins image
viewer = napari.view_image(image, name='Vesicles', rgb=False)

# add the labels
label_layer = viewer.add_labels(label_image, name='segmentation')

napari.run()

### Analyze Segmentation ###

In [12]:
# Create the properties dictionary
properties = regionprops_table(
    label_image, properties=('label', 'bbox', 'perimeter', 'area')
)

In [13]:
# Define measurement: Circularity
def circularity(perimeter, area):
    """Calculate the circularity of the region

    Parameters
    ----------
    perimeter : float
        the perimeter of the region
    area : float
        the area of the region

    Returns
    -------
    circularity : float
        The circularity of the region as defined by 4*pi*area / perimeter^2
    """
    circularity = 4 * np.pi * area / (perimeter ** 2)

    return circularity

In [14]:
properties['circularity'] = circularity(
    properties['perimeter'], properties['area']
)

In [15]:
# Define bounding boxes
def make_bbox(bbox_extents):
    """Get the coordinates of the corners of a
    bounding box from the extents

    Parameters
    ----------
    bbox_extents : list (4xN)
        List of the extents of the bounding boxes for each of the N regions.
        Should be ordered: [min_row, min_column, max_row, max_column]

    Returns
    -------
    bbox_rect : np.ndarray
        The corners of the bounding box. Can be input directly into a
        napari Shapes layer.
    """
    minr = bbox_extents[0]
    minc = bbox_extents[1]
    maxr = bbox_extents[2]
    maxc = bbox_extents[3]

    bbox_rect = np.array(
        [[minr, minc], [maxr, minc], [maxr, maxc], [minr, maxc]]
    )
    bbox_rect = np.moveaxis(bbox_rect, 2, 0)

    return bbox_rect

In [ ]:
# Create the bounding box rectangles
bbox_rects = make_bbox([properties[f'bbox-{i}'] for i in range(4)])

In [17]:
# Specify the display parameters for the text
text_parameters = {
    'string': 'label: {label}\ncirc: {circularity:.2f}',
    'size': 12,
    'color': 'green',
    'anchor': 'upper_left',
    'translation': [-3, 0],
}

In [19]:
# initialise viewer with coins image
viewer = napari.view_image(image, name='vesicles', rgb=False)

# add the labels
label_layer = viewer.add_labels(label_image, name='segmentation')

napari.run()

In [ ]:
shapes_layer = viewer.add_shapes(
        face_color='transparent',
        edge_color='green',
        name='bounding box'
    )

In [22]:
shapes_layer = viewer.add_shapes(
        face_color='transparent',
        edge_color='green',
        properties=properties,
        text=text_parameters,
        name='bounding box'
    )

ValueError: Length of values (294) does not match length of index (0)

In [ ]:
# create the properties dictionary
properties = {
    'label': label,
    'circularity': circularity,
}

In [23]:
text_parameters = {
    'string': 'label: {label}\ncirc: {circularity:.2f}',
    'size': 12,
    'color': 'green',
    'anchor': 'upper_left',
    'translation': [-3, 0]
}

label: 1
circ: 0.83

In [ ]:
# create the properties dictionary
properties = {
    'label': label,
    'circularity': circularity,
}

# specify the display parameters for the text
text_kwargs = {
    'string': 'label: {label}\ncirc: {circularity:.2f}',
    'size': 12,
    'color': 'green',
    'anchor': 'upper_left',
    'translation': [-3, 0]
}

# initialise viewer with coins image
viewer = napari.view_image(image, name='coins', rgb=False)

# add the labels
label_layer = viewer.add_labels(label_image, name='segmentation')

shapes_layer = viewer.add_shapes(
    bbox_rects,
    face_color='transparent',
    edge_color='green',
    properties=properties,
    text=text_parameters,
    name='bounding box'
)

napari.run()